In [1]:
@time using Cxx

  3.737468 seconds (3.58 M allocations: 211.529 MiB, 3.09% gc time)


In [2]:
@time using CxxROS

  3.934349 seconds (1.68 M allocations: 86.745 MiB, 0.66% gc time)


In [3]:
#reload("CxxROS")

In [4]:
using TimeZones

`RosTime` converts `time()::Float64` to `ros::Time` object, and converts to `DateTime`

In [5]:
#t = @cxx ros::Time(time())
#t0 = time()
#t = icxx"ros::Time((double)$t0);"

#t = convert(cxxt"ros::Time", time())
#t = cxxt"ros::Time"(time())

t = RosTime(time())

(class ros::Time) {
}


In [6]:
#icxx"$t.toSec();"
get(t)

1.52395004820333e9

Julia's `DateTime` is anbiguous in Time Zone (UTC vs. local zone). So, here converts to `ZonedDateTime` object in `localzone()`

In [8]:
convert(ZonedDateTime,t)
#ZonedDateTime(t)

2018-04-17T16:27:28+09:00

example to handle timestamp of ros messages

In [9]:
msg = @cxx nav_msgs::Odometry();

In [10]:
t = RosTimeR(icxx"$msg.header.stamp;")

(class ros::Time &) {
}


In [11]:
#sec = time()
#icxx"$t.fromSec((double) $sec);"

set(t, time())

(class ros::Time &) {
}


In [12]:
#icxx"$t.toSec();"
get(t), get(icxx"$msg.header.stamp;")

(1.523950231666337e9, 1.523950231666337e9)

In [13]:
ZonedDateTime(t), ZonedDateTime(icxx"$msg.header.stamp;")

(2018-04-17T16:30:31+09:00, 2018-04-17T16:30:31+09:00)

experimental implementation of interface to `ros::spin()`, compatible with Jupyter-interaction

In [14]:
# 1st args will create ros::Rate(#1)
spin = RosSpin(1)

LoadError: [91mUnrecognized C++ Exception (N3ros27TimeNotInitializedExceptionE)[39m

..., need to initialize

In [15]:
@cxx ros::init(Int32(1), pointer([pointer("test")]), pointer("test"))
nh = @cxx ros::NodeHandle();

In [16]:
# 1st args will create ros::Rate(#1)
spin = RosSpin(1)

CxxROS.RosSpin(false, nothing, (class ros::Rate) {
}
)

In [18]:
start(spin)

Task (runnable) @0x00007f65911c07f0

In [19]:
istaskdone(spin.task)

false

In [20]:
stop(spin)

false

In [22]:
istaskdone(spin.task), spin.task

(true, Task (done) @0x00007f65911c07f0)

In [23]:
start(spin)

Task (runnable) @0x00007f659119a230

undocumented `throwto` method could throw exception to the task will terminate it. 
Maybe `@async` decoration is needed to return to the Jupyter.

In [24]:
@async Base.throwto(spin.task, ErrorException("stop"))

got exception: stop

Task (runnable) @0x00007f6591190550

In [25]:
istaskdone(spin.task), spin.task

(true, Task (done) @0x00007f659119a230)

### the rest is workaround

In [102]:
cxxt"ros::TimeBase<$T,$D>" where {T, D}

Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("ros::TimeBase")},Tuple{T,D}},(false, false, false)},N} where N where D where T

In [107]:
d =icxx"ros::Duration(3.);"

(class ros::Duration) {
}


In [109]:
n = time()
t = icxx"ros::Time((double)$n);"

(class ros::Time) {
}


In [111]:
icxx"($t+$d).toSec();"

1.523852696893647e9

In [4]:
cxxt"ros::Time"

Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N

In [5]:
cxxt"ros::Time&"

Cxx.CppRef{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)}

In [22]:
t = icxx"ros::Time();"

(class ros::Time) {
}


In [23]:
icxx"$t.isValid();"

false

In [93]:
t0 = time()

1.523851850941084e9

In [135]:
#t1 = icxx"ros::Time((double)$t0);"
# modf: (t::Float64) -> (UInt32(trunc(rem(t,one(t))*1e9)), UInt32(trunc(t)))
#       (t::Float64) -> (UInt32(round((t-floor(t))*1e9)), UInt32(floor(t)))
nsec, sec = ((t::Float64) -> (UInt32(round((t-floor(t))*1e9)), UInt32(floor(t))))(t0)
t1 = icxx"ros::Time($sec,$nsec);"

(class ros::Time) {
}


In [136]:
icxx"$t1.toSec();"

1.523851850941084e9

In [143]:
t0 = time()
t1 = icxx"ros::Time((double)$t0);"

(class ros::Time) {
}


In [146]:
icxx"$t1.toSec();", t0

(1.523855050629941e9, 1.523855050629941e9)

In [4]:
t = convert(cxxt"ros::Time", time())

(class ros::Time) {
}


In [5]:
t = (cxxt"ros::Time")(time())

(class ros::Time) {
}


In [6]:
icxx"$t.toSec();"

1.523866358260103e9

In [7]:
t2 = Dates.epochms2datetime(t)

2018-04-16T08:12:38.26

In [7]:
typeof(t2)

DateTime

In [8]:
t3 = Dates.datetime2epochms(t2)

63691085558260

In [9]:
typeof(t3)

Int64

In [10]:
Dates.unix2datetime(icxx"$t.toSec();")

2018-04-16T08:12:38.26

In [12]:
t3 = ZonedDateTime(t2, tz"UTC")

2018-04-16T08:12:38.26+00:00

In [13]:
t4 = astimezone(t3, tz"Asia/Tokyo")

2018-04-16T17:12:38.26+09:00

In [20]:
typeof(t4)

TimeZones.ZonedDateTime

In [18]:
ZonedDateTime((cxxt"ros::Time")(time()))

2018-04-16T17:16:43.238+09:00

## DateTime and ZonedDateTime

``now()`` returns ``DateTime`` object in localzone

In [19]:
now()

2018-04-16T17:17:19.968

In [21]:
using TimeZones

In [17]:
ZonedDateTime(now(), localzone())

2018-04-16T17:16:31.565+09:00

### epoch time

[Date and DateTime · The Julia Language](https://docs.julialang.org/en/stable/manual/dates/)

[DateTime bug - Julia 0.6RC3 - Usage - JuliaLang](https://discourse.julialang.org/t/datetime-bug-julia-0-6rc3/4278)

In [24]:
DateTime(Dates.UTM(0))

0000-12-31T00:00:00

In [25]:
Dates.datetime2epochms(DateTime(Dates.UTM(0)))

31536000000

In [26]:
Dates.epochms2datetime(0)

0000-01-01T00:00:00

unix epch is exact, and ``time()`` returns seconds from unix epoch in UTC

In [27]:
Dates.unix2datetime(0)

1970-01-01T00:00:00

In [28]:
time()

1.523867463891154e9

useful representation coms from this

In [31]:
dt = Dates.unix2datetime(time())

2018-04-16T08:36:59.446

In [36]:
astimezone(ZonedDateTime(dt, tz"UTC"), localzone())

2018-04-16T17:36:59.446+09:00

but this going to missunderstandings...

In [33]:
Dates.datetime2epochms(dt)

63691087019446

In [10]:
t = (cxxt"ros::Time")()

LoadError: [91mMethodError: no method matching Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N()[39m

In [13]:
mutable struct RosType
  cppname::String
  cppval
  cppref
  cppvalorref
  function RosType(cppname)
    v = Expr(:macrocall, Symbol("@cxxt_str"), string(cppname))
    r = Expr(:macrocall, Symbol("@cxxt_str"), string(cppname, "&"))
    cppval = @eval $v
    cppref = @eval $r
    b =  Expr(:macrocall, Symbol("@icxx_str"), string(cppname, "();"))
    @eval (::Type{$cppval})() = $b
    new(cppname, cppval, cppref, Union{cppval,cppref})
  end
end

In [14]:
t = RosType("ros::Time")

RosType("ros::Time", Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N, Cxx.CppRef{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)}, Union{Cxx.CppRef{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)}, Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N})

In [19]:
rt = (t.cppval)()

(class ros::Time) {
}


In [22]:
typeof(rt) <: t.cppvalorref

true

In [26]:
CxxStd.StdVector

Union{Cxx.CppRef{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{T,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::allocator")},Tuple{T}},(false, false, false)}}},(false, false, false)}, Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{T,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::allocator")},Tuple{T}},(false, false, false)}}},(false, false, false)},N} where N} where T

In [27]:
using CxxStd: StdVector

In [38]:
Int32[1,2,3]

3-element Array{Int32,1}:
 1
 2
 3

In [43]:
convert(StdVector{Int32},Int32[1,2,3])

LoadError: [91mMethodError: Cannot `convert` an object of type Array{Int32,1} to an object of type Union{Cxx.CppRef{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{Int32,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::allocator")},Tuple{Int32}},(false, false, false)}}},(false, false, false)}, Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{Int32,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::allocator")},Tuple{Int32}},(false, false, false)}}},(false, false, false)},N} where N}
This may have arisen from a call to the constructor Union{Cxx.CppRef{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{Int32,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::allocator")},Tuple{Int32}},(false, false, false)}}},(false, false, false)}, Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{Int32,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::alloc

In [44]:
t = Int32
convert(Type(cxxt"std::vector<$t>"), Int32[1,2,3])

(class std::vector<int, class std::allocator<int> >) {
}


In [46]:
v = icxx"std::vector<int>();"

(class std::vector<int, class std::allocator<int> >) {
}


In [48]:
convert(typeof(v), Int32[1,2,3])

LoadError: [91mMethodError: Cannot `convert` an object of type Array{Int32,1} to an object of type Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{Int32,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::allocator")},Tuple{Int32}},(false, false, false)}}},(false, false, false)},24}
This may have arisen from a call to the constructor Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{Int32,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::allocator")},Tuple{Int32}},(false, false, false)}}},(false, false, false)},24}(...),
since type constructors fall back to convert methods.[39m

In [49]:
typeof(v)

Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::vector")},Tuple{Int32,Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("std::allocator")},Tuple{Int32}},(false, false, false)}}},(false, false, false)},24}

In [52]:
t = Int32
typeof(v) == Type(cxxt"std::vector<$t>")

false

In [11]:
icxx"($t.cppname)();"

:2:13: error: member access into incomplete type '(anonymous class)'
(__juliavar1.cppname)();
            ^
note: definition of '(anonymous class)' is not complete until the closing '}'
:2:13: error: member access into incomplete type '(anonymous class)'
(__juliavar1.cppname)();
            ^
note: definition of '(anonymous class)' is not complete until the closing '}'


LoadError: [91mA failure occured while parsing the function body[39m

In [6]:
typeof(t)

UnionAll

In [30]:
t = @eval Expr(:macrocall, Symbol("@cxxt_str"),  string("ros::Time", "&"))

:(@cxxt_str "ros::Time&")

In [31]:
@eval $t

Cxx.CppRef{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)}

In [40]:
  function x(name)
    val = Expr(:macrocall, Symbol("@cxxt_str"), string(name))
    ref = Expr(:macrocall, Symbol("@cxxt_str"), string(name, "&"))
    valorref = "H"
    @eval $(val)
  end

x (generic function with 1 method)

In [41]:
x("ros::Time")

Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N

In [51]:
t.val

Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N

In [52]:
t.ref

Cxx.CppRef{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)}

In [53]:
t.valorref

Union{Cxx.CppRef{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)}, Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N}

In [54]:
t.val="H"

"H"

In [55]:
t.val

"H"

In [56]:
t.ref

Cxx.CppRef{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)}

In [57]:
t.valorref

Union{Cxx.CppRef{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)}, Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N}

In [58]:
:t

:t

In [60]:
t() = "H"

LoadError: [91mcannot define function t; it already has a value[39m

In [ ]:
rt = icxx"ros::Time();"

In [67]:
(Type{rt})()

LoadError: [91mMethodError: no constructors have been defined for Type{Cxx.CppValue{Cxx.CxxQualType{Cxx.CppBaseType{Symbol("ros::Time")},(false, false, false)},N} where N}[39m